In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[17]:


import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy

from scipy.optimize import curve_fit


# In[18]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.get_virtual_cs()
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs for cross_section in self.sig_t]
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)    
        
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0. 
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  

class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
         for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
             #   print("   " + str(i))
                if boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = self.surfaces[i].distance(particle)
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):

        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)


# In[19]:


def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))


# In[20]:


class Flux_estimator:
    def __init__(self, universe):
        self.boundaries = universe
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()


# In[21]:


import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


# In[22]:


def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path

def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)
    
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    virtual_cs = c_s.virtual[energy_group_idx]
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False
    
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)
    

def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision

def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

def process_one_particle_history(particle, universe, c_s, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        free_path = get_free_path(particle, c_s)
        set_random_direction(particle)
        move_particle(particle, free_path)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                
        for k in estimators:
            k.add_collision(particle)
            
        if is_collision_virual(particle, c_s):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, c_s)
        
            
    return particle, sum_collisions

def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        
        
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))
                    
                    
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])
                
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")               


# In[23]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]/(sig_t * sum(weight_previos))
        estimator.collision_sum.append(0.)


# In[24]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std     


# In[25]:


def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))


# In[26]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
            
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, c_s, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 

        batch_particles = normalise_weights(batch_particles, initial_size) 
        

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators


# In[27]:


cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]

cs_production_neutrons_h2o = [0.0]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

h2o_cs_1_4 = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]


# In[28]:


def get_quarter_cylinder_estimators(radius):
    
    estimators = []
    
    points_to_divide = [0.5]
    epsilon = 0.1
    
    
    cylinder_2 = ZCylinder(0, 0, epsilon)
    plane_5 = Plane(0, 0, 1, 1000)
    plane_6 = Plane(0, 0, 1, -1000)

    surfaces = [cylinder_2, plane_5, plane_6]
    signs = [-1, -1, +1]

    cylinder = Cell(surfaces, signs)
    estimators.append(Flux_estimator(cylinder))
    
    for point in points_to_divide:
        
        if point < 1:
            cylinder_1 = ZCylinder(0, 0, point * radius + epsilon)        
        else:
            cylinder_1 = ZCylinder(0, 0, point * radius)
            
        cylinder_2 = ZCylinder(0, 0, point * radius - epsilon)
        

        plane_5 = Plane(0, 0, 1, 100000)
        plane_6 = Plane(0, 0, 1, -100000)

        surfaces = [cylinder_1, cylinder_2, plane_5, plane_6]
        signs = [-1, +1, -1, +1]

        cylinder = Cell(surfaces, signs)
        estimators.append(Flux_estimator(cylinder))
    

    cylinder_1 = ZCylinder(0, 0, radius)
            
    cylinder_2 = ZCylinder(0, 0, radius - 0.02)
        

    plane_5 = Plane(0, 0, 1, 100000)
    plane_6 = Plane(0, 0, 1, -100000)

    surfaces = [cylinder_1, cylinder_2, plane_5, plane_6]
    signs = [-1, +1, -1, +1]

    cylinder = Cell(surfaces, signs)
    estimators.append(Flux_estimator(cylinder))    
        
        
    return estimators


# In[29]:


## Infinite cylinder parallel to z-axis

class ZCylinder:
    def __init__(self, x_0, y_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.r = r
        
    def distance(self, particle):
    
        a_ = particle.direction.tetta_x * particle.direction.tetta_x + (particle.direction.tetta_y * particle.direction.tetta_y )
        
        ## If a=0a 0a=0, this means the particle is parallel to the cylinder and will thus never intersect it
        if (abs(a_) < 1e-9):
            return -1
        
        c_ = ((particle.coordinates.x - self.x_0) * (particle.coordinates.x - self.x_0) + 
              (particle.coordinates.y - self.y_0) * (particle.coordinates.y - self.y_0) - self.r * self.r)

        k_ = (particle.coordinates.x -self.x_0) * particle.direction.tetta_x + (particle.coordinates.y -self.y_0) * particle.direction.tetta_y
        
        dec = k_ * k_ - a_ * c_
        
        if dec < 0:
            return -1
        
        dist_1 = (-k_ + math.sqrt(k_ * k_ - a_ * c_))/a_
        dist_2 = (-k_ - math.sqrt(k_ * k_ - a_ * c_))/a_
         
        distance = min(dist_1, dist_2)
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1
        
        
    def get_xml(self):

        obj_xml = " ZCylinder with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " r = "+ str(r) 
        
        return obj_xml


# In[33]:


test_number_of_particles = 20
test_number_interations = 1000
test_number_inactive = 200


# In[34]:


cylinder_estimators = get_quarter_cylinder_estimators(4.279960)
print(len(cylinder_estimators))


# In[35]:


test_cylinder = ZCylinder(0, 0, 4.279960)

plane_5 = Plane(0, 0, 1, 1000)
plane_6 = Plane(0, 0, 1, -1000)

surfaces = [test_cylinder, plane_5, plane_6]
signs = [-1, -1, +1]
boundaries_type = ["black", "reflective", "reflective"]

cylinder = Cell(surfaces, signs)

cylinder.set_boundaries_type(boundaries_type)

cylinder.set_box_sizes(-1000, -4.279960, -1000, 1000, 4.2799605, 1000 )
cylinder.set_zero_point(0., 0., 0.)


# In[36]:



k_effective, k_effective_exp, k_effective_std, estimators = simulation_black_boundaries(cylinder,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4, cylinder_estimators)





difference = (k_effective_exp[-1] - 1.) * 100000
standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


# In[66]:


def get_std_flux(estimators):
    
    current_std = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        current_std.append(get_std(array_without_zeros))
    
    return current_std  


# In[67]:


def get_average_number_of_collision(estimators):
    
    average_number_of_collision = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[20:len(current_estimator.collision_sum) - 1]
        average_number_of_collision.append((sum(array_without_zeros)) / len(array_without_zeros))        
            
    return average_number_of_collision
    


# In[68]:


average_sum_of_collision = get_average_number_of_collision(estimators)





def divide_estimators_square(average_number_of_collision, radius, epsilon):
    average_number_of_collision[0] /= np.pi * epsilon * epsilon
    
    average_number_of_collision[1] /= np.pi * ((radius * 0.5 + epsilon) * (radius * 0.5 + epsilon) -
                                            (radius * 0.5 - epsilon) * (radius * 0.5 - epsilon))
    
    average_number_of_collision[2] /= np.pi * ((radius) * (radius) -
                                            (radius - 0.02) * (radius - 0.02))
    
    
    
    result_estimators = [ average_number_of_collision[1]/
                         average_number_of_collision[0], average_number_of_collision[2]/ average_number_of_collision[0]]
    
    return result_estimators, average_number_of_collision[0]


# In[71]:


def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()


# In[72]:


average_sum_of_collision = get_average_number_of_collision(estimators)


# In[73]:


normalise_flux, zero_point = divide_estimators_square(average_sum_of_collision, 4.279960, 0.1)




flux_std = get_std_flux(estimators)



test_flux = [0.8093, 0.2926]


# In[65]:


for i in range(0, len(test_flux)):
    print(str(((normalise_flux[i] - test_flux[i])/test_flux[i] )* 100) + " % " + str(flux_std[i + 1] * 100000))


# In[40]:


file_name = "result_flux.txt"
save_flux_to_file(estimators, file_name)


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[58]:


file_name = "output"
save_simulation_results_to_file(k_effective, file_name)





3
 num1  1
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
i == 103
i == 104
i == 105
i == 106
i == 107
i == 108
i == 109
i == 110
i == 111
i == 112
i == 113
i == 114
i == 115
i == 116
i == 117
i == 118
i == 119
i == 120
i == 121
i

 keff_cycle , k_effective_exp, std_k_effective 1.001902959722789   0.9976153727794426  0.0019980054997003364
i == 258
 keff_cycle , k_effective_exp, std_k_effective 0.9939551105169978   0.9975522648094004  0.001964268954229279
i == 259
 keff_cycle , k_effective_exp, std_k_effective 1.0107635781943498   0.997776185375247  0.0019436309663504506
i == 260
 keff_cycle , k_effective_exp, std_k_effective 1.0407675482874226   0.9984927080904499  0.0020408779377953054
i == 261
 keff_cycle , k_effective_exp, std_k_effective 1.003045685392473   0.9985673470626142  0.0020085294033357006
i == 262
 keff_cycle , k_effective_exp, std_k_effective 1.0280831844261884   0.9990434089555751  0.002032409928375482
i == 263
 keff_cycle , k_effective_exp, std_k_effective 0.9907497786244968   0.998911764029685  0.00200421742430452
i == 264
 keff_cycle , k_effective_exp, std_k_effective 0.9974115631678794   0.9988883233912192  0.001972792237805616
i == 265
 keff_cycle , k_effective_exp, std_k_effective 0.99677775

 keff_cycle , k_effective_exp, std_k_effective 0.9548762703206389   0.9978727672617121  0.001401978402254285
i == 328
 keff_cycle , k_effective_exp, std_k_effective 0.9677823836838582   0.9976376861400101  0.001410707254504664
i == 329
 keff_cycle , k_effective_exp, std_k_effective 0.9988073028046185   0.9976467529358598  0.001399758186011098
i == 330
 keff_cycle , k_effective_exp, std_k_effective 0.9922864831012329   0.997605520090978  0.0013895609725201972
i == 331
 keff_cycle , k_effective_exp, std_k_effective 0.986526644089756   0.9975209485184496  0.0013815038790743936
i == 332
 keff_cycle , k_effective_exp, std_k_effective 0.9887876750097011   0.9974547873555045  0.0013725934569890933
i == 333
 keff_cycle , k_effective_exp, std_k_effective 0.9733559089712174   0.9972735927811867  0.0013742318772354944
i == 334
 keff_cycle , k_effective_exp, std_k_effective 0.9859080241054141   0.9971887751045018  0.0013665725468218156
i == 335
 keff_cycle , k_effective_exp, std_k_effective 0.9923

 keff_cycle , k_effective_exp, std_k_effective 0.9806294823153213   0.9991542923859494  0.001125339948201017
i == 398
 keff_cycle , k_effective_exp, std_k_effective 0.9881311161370198   0.9990986197786317  0.0011210252539754595
i == 399
 keff_cycle , k_effective_exp, std_k_effective 0.9977544427095071   0.9990918651199929  0.0011153981883746222
i == 400
 keff_cycle , k_effective_exp, std_k_effective 1.001845511641885   0.9991056333526024  0.0011098925856461441
i == 401
 keff_cycle , k_effective_exp, std_k_effective 1.005069297273049   0.9991353033223559  0.0011047554163275662
i == 402
 keff_cycle , k_effective_exp, std_k_effective 1.004943347415467   0.9991640560158861  0.0010996486902841987
i == 403
 keff_cycle , k_effective_exp, std_k_effective 0.9937493241748221   0.9991373824600187  0.0010945433530652085
i == 404
 keff_cycle , k_effective_exp, std_k_effective 0.9732179054771718   0.999010326200299  0.0010965505461438293
i == 405
 keff_cycle , k_effective_exp, std_k_effective 0.9859

 keff_cycle , k_effective_exp, std_k_effective 1.006271075865384   0.9992591204232034  0.0009053664890653739
i == 468
 keff_cycle , k_effective_exp, std_k_effective 1.0313795637924552   0.9993789728238349  0.0009099098852382643
i == 469
 keff_cycle , k_effective_exp, std_k_effective 0.9863109034200549   0.9993303926401778  0.0009078217751912846
i == 470
 keff_cycle , k_effective_exp, std_k_effective 1.000724509493383   0.9993355560359305  0.0009044679610936179
i == 471
 keff_cycle , k_effective_exp, std_k_effective 0.9919659569155888   0.9993083619432355  0.0009015344980803731
i == 472
 keff_cycle , k_effective_exp, std_k_effective 1.002269832314842   0.9993192497019545  0.0008982799036596237
i == 473
 keff_cycle , k_effective_exp, std_k_effective 1.0192183579392666   0.9993921402083183  0.0008979467724849066
i == 474
 keff_cycle , k_effective_exp, std_k_effective 1.0058401659185663   0.9994156731488668  0.0008949730402016687
i == 475
 keff_cycle , k_effective_exp, std_k_effective 0.97

...

 keff_cycle , k_effective_exp, std_k_effective 0.9844875186910569   0.9995684756999452  0.0006924427676170941
i == 678
 keff_cycle , k_effective_exp, std_k_effective 0.9612072585036117   0.9994882221074843  0.0006956374383747534
i == 679
 keff_cycle , k_effective_exp, std_k_effective 1.0119833041271193   0.9995143078737049  0.0006946735965817798
i == 680
 keff_cycle , k_effective_exp, std_k_effective 0.9767280667103013   0.9994668365379478  0.0006948483426731655
i == 681
 keff_cycle , k_effective_exp, std_k_effective 0.9918389711452748   0.999450978189938  0.0006935835659715302
i == 682
 keff_cycle , k_effective_exp, std_k_effective 1.0072461782219224   0.9994671508041123  0.0006923320189565238
i == 683
 keff_cycle , k_effective_exp, std_k_effective 1.0061800846190345   0.9994810492178078  0.0006910369117946872
i == 684
 keff_cycle , k_effective_exp, std_k_effective 1.0273569663740065   0.9995386440879651  0.0006920086052559509
i == 685
 keff_cycle , k_effective_exp, std_k_effective 1.

 keff_cycle , k_effective_exp, std_k_effective 1.0219450462358617   0.9997389770243726  0.0006580750497721053
i == 748
 keff_cycle , k_effective_exp, std_k_effective 1.0093733047667395   0.9997565579144134  0.0006571083150932264
i == 749
 keff_cycle , k_effective_exp, std_k_effective 1.004807463651469   0.9997657581070128  0.0006559748249518299
i == 750
 keff_cycle , k_effective_exp, std_k_effective 0.994100047889752   0.9997554568157088  0.0006548620843329222
i == 751
 keff_cycle , k_effective_exp, std_k_effective 0.9993333222993137   0.9997546906913596  0.0006536729554096235
i == 752
 keff_cycle , k_effective_exp, std_k_effective 1.007175483761398   0.9997681341570662  0.0006526261665543487
i == 753
 keff_cycle , k_effective_exp, std_k_effective 0.9918650685637285   0.9997538428992121  0.0006516016821567366
i == 754
 keff_cycle , k_effective_exp, std_k_effective 1.0022991622917794   0.999758437338549  0.0006504406691797461
i == 755
 keff_cycle , k_effective_exp, std_k_effective 0.981

 keff_cycle , k_effective_exp, std_k_effective 0.9786628810982074   0.9997573130620373  0.0006165075504310358
i == 818
 keff_cycle , k_effective_exp, std_k_effective 1.0263544670149123   0.9998003505279804  0.0006170119494816356
i == 819
 keff_cycle , k_effective_exp, std_k_effective 0.9775324523645023   0.9997643765406404  0.00061706386460418
i == 820
 keff_cycle , k_effective_exp, std_k_effective 1.005234200312966   0.9997731988370474  0.0006161309622526657
i == 821
 keff_cycle , k_effective_exp, std_k_effective 0.9915607208127886   0.9997599742347539  0.0006152801415212887
i == 822
 keff_cycle , k_effective_exp, std_k_effective 1.0097386619197108   0.999776017141  0.000614499602625945
i == 823
 keff_cycle , k_effective_exp, std_k_effective 0.9847555313153726   0.9997519072119059  0.0006139860095588181
i == 824
 keff_cycle , k_effective_exp, std_k_effective 0.9896779825712553   0.9997357631019048  0.0006132138181525107
i == 825
 keff_cycle , k_effective_exp, std_k_effective 0.9905612

 keff_cycle , k_effective_exp, std_k_effective 1.0195623291651836   0.9995573400890047  0.0005842607057924572
i == 888
 keff_cycle , k_effective_exp, std_k_effective 1.0160605451717875   0.9995813273056947  0.0005839037862974374
i == 889
 keff_cycle , k_effective_exp, std_k_effective 1.006957065665468   0.9995920322960572  0.0005831539691594874
i == 890
 keff_cycle , k_effective_exp, std_k_effective 0.9999706695841835   0.9995925810457501  0.0005823084637248531
i == 891
 keff_cycle , k_effective_exp, std_k_effective 1.0102993163375795   0.9996080755975472  0.0005816715574258614
i == 892
 keff_cycle , k_effective_exp, std_k_effective 1.0340164880812728   0.9996577987369746  0.0005829548222947253
i == 893
 keff_cycle , k_effective_exp, std_k_effective 0.977237307607344   0.9996254459359216  0.0005830113720660973
i == 894
 keff_cycle , k_effective_exp, std_k_effective 1.0073471112870667   0.9996365722548716  0.0005822770041088844
i == 895
 keff_cycle , k_effective_exp, std_k_effective 1.0

 keff_cycle , k_effective_exp, std_k_effective 0.9902172463701826   0.9997711390395208  0.0005592674104931314
i == 958
 keff_cycle , k_effective_exp, std_k_effective 0.9909220488134489   0.9997594647780088  0.0005586510965536009
i == 959
 keff_cycle , k_effective_exp, std_k_effective 1.0080191717020488   0.9997703471323224  0.0005580206976571382
i == 960
 keff_cycle , k_effective_exp, std_k_effective 0.9941826385799571   0.9997629948842272  0.0005573344730335334
i == 961
 keff_cycle , k_effective_exp, std_k_effective 0.9804350463827666   0.999737596791584  0.0005571807844162748
i == 962
 keff_cycle , k_effective_exp, std_k_effective 1.0065925155887563   0.9997465927480107  0.0005565218084127155
i == 963
 keff_cycle , k_effective_exp, std_k_effective 1.0060637389145786   0.9997548721007847  0.0005558536066862999
i == 964
 keff_cycle , k_effective_exp, std_k_effective 0.9862362555646204   0.9997371775765228  0.000555407506075126
i == 965
 keff_cycle , k_effective_exp, std_k_effective 0.9